<a href="https://colab.research.google.com/github/pchodon/Amegashita/blob/Feature-Integration/main_v21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import multiprocessing
import time
from csv import reader
from google.colab import files
import pandas as pd
import threading
import os
import tensorflow as tf
import numpy as np
import sys


from tensorflow import feature_column
#from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


No_of_records = 4
Parameters = 4
No_of_elements = 10

x1 = tf.zeros([No_of_records,Parameters,No_of_elements])
np.b =np.zeros((No_of_records,Parameters,No_of_elements))
ta = tf.TensorArray(tf.float32, size=sys.getsizeof(x1), dynamic_size=True, clear_after_read=False)
rank_3_tensor= tf.convert_to_tensor(np.b, dtype=tf.float32)



sensory_1_resp = 1  # default 1 Sec
sensory_2_resp = 1  # default 1 Sec
temporal_resp =  1  # default 1 Sec
thalamus_resp =  1  # default 1 Sec
reset = 0
forever = 1
max_sensory_stream = 20
Max_lapse = 8
mTrue = 1
mFalse = 0
w, h = 10, 10;
hipp_first = False
stop_emulation = 0
idle_window = 10
window_start = 0

row,column = (4,10)

# row 1 = time
# row 2 = sensor_1
# row_3 = sensor_1
# row_4 = learning

#101213141516171819
#20212223242526272829
#3031323334353637383930

TIME = 1
SENSOR_1 = 2
SENSOR_2 = 3
LEARNING = 4


# send_1_sensor,recv_1_sensor
# send_2_sensor,recv_2_senor

def sensory_1(send_1_sensor,recv_1_sensor,parent_conn,msg):
   
  data = []
  with open('/content/drive/MyDrive/sense_1.csv', 'r') as data:
    df_sensory_1 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_1.loc[i]
   send_1_sensor.send(msg[0])
   time.sleep(sensory_1_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def sensory_2(send_2_sensor,recv_2_sensor,parent_conn,msg):
  
  data = []
  with open('/content/drive/MyDrive/sense_2.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_2.loc[i]
   send_2_sensor.send(msg[0])
   time.sleep(sensory_2_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def hippocampus(xparent_conn,xchild_conn,msg):
    while(1):
      sense = xparent_conn.recv()
      if((sense == 4)):
          print("Hipp Recvd :  ",sense)
          return
      continue


def Idle(idle_state_send,msg):
  data = []
  with open('/content/drive/MyDrive/sense_3.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  time.sleep(0)
  msg =9
  msg = df_sensory_2.loc[i]
  idle_state_send.send(msg[0])

  while(True):
       time.sleep(idle_window)
       msg =9
       msg = df_sensory_2.loc[i] -1
       send_2_sensor.send(msg[0])
       send_1_sensor.send(msg[0])
       

     


def thalamus(b,idle_state_recv,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn,xchild_conn,msg):
  maxim = 10
  lapse = 0
  window_start = 0
  window = False
  source = False
  sense = 0
  sensor_1,sensor_2 = [3,10]
  # row 1 time
  # row 2 sensor_1
  
  # row 3 sensor_2
  # Episode is 10 frames
  think_pause = 0
 
  episode_count = 0
  experience_count = 0

  if(window_start == 0):
        sense = idle_state_recv.recv()
        print(sense)
        if((sense == 9)):
         print("________Window starts here____________________")
         No_of_records = 4
         Parameters = 4
         No_of_elements = 10
         np.b =np.zeros((No_of_records,Parameters,No_of_elements))
         No_of_records = 4
         Parameters = 4
         No_of_elements = 10
         episode_count = 0
         window_start = 1
         experience_count = 0
         
  while(1):
    #print("Thalamus while stating")
    #time.sleep(temporal_resp)
    sense_1 = recv_1_sensor.recv()
    #print('sense 1 : ',sense_1)
    sense_2 = recv_2_sensor.recv()
    #print('sense 2 : ',sense_2)
    
    print("Exp Epi Count",sense_1,sense_2,experience_count,episode_count,)


    if((sense_2 == 8)): 
          print("__________Start thinking")
          rank_3_tensor= tf.convert_to_tensor(np.b, dtype=tf.float32)
          print(rank_3_tensor) 
          think_pause == 0
          episode_count = 0
          continue
          experience_count = experience_count + 1
                       
    if((sense_1 == '1') or (sense_1 =='0')):
         np.b[experience_count][0][episode_count] = sense_1

    if((sense_2 == '1') or (sense_2 =='0')):  
         np.b[experience_count][1][episode_count] = sense_2
     
    if((sense_2 == 'x') or (sense_2 =='x')):  
     print("__________No action__________________")
     continue 
    episode_count +=1
    if(episode_count >= 10):
      experience_count = experience_count + 1
      episode_count = 0

    continue

         
#continue


# As sensory input is received
# Start a timer window  
# Reset the window e experiences no sensory inputs


if __name__ == "__main__":
  print("System Starting......")
  msg = 0

        # creating common pipe name
  parent_conn, child_conn = multiprocessing.Pipe()
  send_1_sensor,recv_1_sensor = multiprocessing.Pipe()
  send_2_sensor,recv_2_sensor = multiprocessing.Pipe()
  idle_state_send,idle_state_recv = multiprocessing.Pipe()
  xparent_conn, xchild_conn = multiprocessing.Pipe()
  timer = multiprocessing.Pipe()

  p1 = multiprocessing.Process(target=sensory_1, args=(send_1_sensor,recv_1_sensor,parent_conn,msg))
  p2 = multiprocessing.Process(target=sensory_2, args=(send_2_sensor,recv_2_sensor,parent_conn,msg))
  p3 = multiprocessing.Process(target=thalamus, args=(np.b,idle_state_recv,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn, xchild_conn,msg))
  p4 = multiprocessing.Process(target=hippocampus, args=(xparent_conn,xchild_conn,msg))
  p5 = multiprocessing.Process(target=Idle, args=(idle_state_send,msg))



       # running processes
  p1.start()
  p2.start()
  p3.start()
  p4.start()
  p5.start()
 
  time.sleep(100)
  
  print("Starting new cycle")
  p5.join()
  p4.join()
  p3.join()
  p1.join()
  p2.join()

  time.sleep(2)

print("Starting new episode")
#333444

System Starting......
9
________Window starts here____________________
Exp Epi Count 0 1 0 0
Exp Epi Count 0 1 0 1
Exp Epi Count 0 1 0 2
Exp Epi Count 0 1 0 3
Exp Epi Count 1 0 0 4
Exp Epi Count 1 0 0 5
Exp Epi Count 1 0 0 6
Exp Epi Count 1 0 0 7
Exp Epi Count 1 0 0 8
Exp Epi Count 1 0 0 9
Exp Epi Count 1 8 10 
__________Start thinking
tf.Tensor(
[[[0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]], shape=(4, 4, 10), dtype=float32)
Exp Epi Count 8 0 1 0
Exp Epi Count x x 1 1
__________No action__